In [72]:
import numpy as np
import os
from os.path import join
import pandas as pd
import torch
from torch import nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from tqdm.notebook import tqdm


In [85]:
PRETRAINED = 'roberta-large'
BATCHSIZE = 32
MAXLENGTH = 32
SEED = 42
TRAIN_RATIO = 0.8
EVAL_RATIO = 0.9
DROPOUT = 0.1
NUM_LABELS = 8
PROJECT_ROOT = os.getcwd()
EPOCHS = 5
LR = 3e-5

use_cuda = torch.cuda.is_available()
use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
device = torch.device('cpu')
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED)
config = AutoConfig.from_pretrained( 'bert-base-uncased', output_hidden_states=True) 


In [74]:
datapath = 'dataset.tsv'
df = pd.read_csv(datapath, sep='\t')


In [ ]:

sample_text = "This is some sample text"
bert_input = tokenizer(sample_text, padding='max_length', max_length=MAXLENGTH, truncation = True, return_tensors='pt')

print(bert_input)
text_passed_in = tokenizer.decode(bert_input.input_ids[0])
print(text_passed_in)

In [86]:
labels = {'Not Dark Pattern':0,
 'Misdirection':1,
 'Forced Action':2,
 'Obstruction':3,
 'Scarcity':4,
 'Sneaking':5,
 'Social Proof':6,
 'Urgency':7}

class DarkPatternDataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = torch.tensor([labels[label] for label in df['Pattern Category']])
        self.texts = [tokenizer(text, padding='max_length',max_length=MAXLENGTH, truncation=True, return_tensors='pt') for text in df['text']]
        #self.texts = [text['input_ids'] for text in self.texts]

    def classes(self):
        return labels
    
    def __len__(self):
        return len(self.labels)
    
    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_labels = self.get_batch_labels(idx)
        return batch_texts, batch_labels

d = DarkPatternDataset(df)


In [77]:

np.random.seed(SEED)
df_train, df_val, df_test = np.split(
    df.sample(frac=1, random_state=SEED),
    [int(TRAIN_RATIO*len(df)), int(EVAL_RATIO*len(df))])


c:\Users\tanwa\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [78]:
class DarkPatternClassifier(nn.Module):

    def __init__(self, dropout = DROPOUT):
        super(DarkPatternClassifier, self).__init__()

        self.model = AutoModelForSequenceClassification.from_pretrained(PRETRAINED, config=config)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, NUM_LABELS)
        self.relu = nn.ReLU()
    
    def forward(self, input_id, mask):

        _, pooled_output = self.model(input_ids = input_id, attention_mask = mask, return_dict = False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer
    
    

In [79]:
train_dataloader = DataLoader(DarkPatternDataset(df_train), batch_size=BATCHSIZE, shuffle=True)
print(len(train_dataloader.dataset))
print(len(train_dataloader.dataset)/BATCHSIZE)
for train_input, train_label in tqdm(train_dataloader):
    train_label = train_label.to(device)
    mask = train_input['attention_mask'].to(device)
    input_id = train_input['input_ids'].squeeze(1).to(device)
    print(input_id.shape[0])
    print(torch.max(mask), torch.min(mask))

1884
942.0


  0%|          | 0/942 [00:00<?, ?it/s]

2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1) tensor(0)
2
tensor(1

In [80]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = DarkPatternDataset(train_data), DarkPatternDataset(val_data)

    train_dataloader = DataLoader(train, batch_size=BATCHSIZE, shuffle=True)
    val_dataloader = DataLoader(val, batch_size=BATCHSIZE)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr = learning_rate)

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()
    
    for epoch_num in range(epochs):        
        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):
            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            batch_loss = criterion(output, train_label.long())
            total_loss_train += batch_loss.item()

            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:
                
                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, val_label.long())
                total_loss_val += batch_loss.item()

                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc
        
        print(
                f"""Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}""")



In [81]:
def evaluate(model, test_data):

    test = DarkPatternDataset(test_data)

    test_dataloader = DataLoader(test, batch_size=BATCHSIZE)

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [82]:
def save(model):
    model_path = join(PROJECT_ROOT, "models\\roberta-large.pth")
    torch.save(model.state_dict(), model_path)

In [87]:
model = DarkPatternClassifier()

checkpoint = torch.load("D:\\Programming\\DPBH-Team_Veritas\\dark_pattern_binary_classifier\\pretrained_models\\roberta-large_4.pth")
model_dict = model.state_dict() 
pretrained_dict = {k: v for k, v in checkpoint.items() if k in model_dict}
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['encoder.layer.4.attention.self.query.weight', 'encoder.layer.2.attention.self.key.bias', 'encoder.layer.6.attention.output.LayerNorm.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.9.output.LayerNorm.weight', 'encoder.layer.4.output.dense.weight', 'encoder.layer.11.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.5.attention.output.LayerNorm.weight', 'encoder.layer.1.attention.self.query.weight', 'encoder.layer.1.intermediate.dense.weight', 'classifier.bias', 'encoder.layer.6.intermediate.dense.weight', 'encoder.layer.6.output.dense.bias', 'encoder.layer.3.attention.self.value.weight', 'encoder.layer.7.intermediate.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.2.attention.self.query.weight', 'embeddings.LayerNorm.bi

<All keys matched successfully>

In [88]:
train(model, df_train, df_val, LR, EPOCHS)

  0%|          | 0/59 [00:00<?, ?it/s]

IndexError: index out of range in self

In [ ]:
evaluate(model, df_test)
save(model)